In [1]:
import json
import os
import pandas as pd
import numpy as np
import preprocessor
from pyvi import ViTokenizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_curve, auc, classification_report, precision_recall_fscore_support
from sklearn.svm import SVC
from sklearn import svm, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score

In [2]:
path_json = '../data/raw_data'

In [3]:
path_urls = os.path.join(path_json,'urls.txt')

In [4]:
with open(path_urls,"rb") as input_file:
    urls = input_file.readlines()

In [5]:
len(urls)

1008

In [6]:
comment_score = {'comment':[]}

In [7]:
for index,url in enumerate(urls):
    try:
        path_index_json = os.path.join(path_json,'{:03d}.json'.format(index))
        with open(path_index_json,"rb") as input_file:
            data = json.load(input_file)
        comment_score['comment'].extend(data['comment'])
    except:
        pass

In [8]:
data_comment = {'comment':[],'score':[]}

In [9]:
for i,text in enumerate(comment_score['comment']):
    data_comment['comment'].append(text['comment'])
    try:
        data_comment['score'].append(text['score'])
    except:
        data_comment['score'].append(-100)

In [10]:
df = pd.DataFrame(data_comment) 

import preprocessor

from pyvi import ViTokenizer
with open("/home/hieuvt/Projects/Crawl_comments/vietnamese-stopwords-dash.txt", encoding="utf8") as input_file:
    x = input_file.readlines()
    removetable = str.maketrans('', '', '\n')
    stopwords = [s.translate(removetable) for s in x]
def prepocess(text):
    return preprocessor.preprocess(text)
def tokenize(text, stopwords):
        text_segmented = ViTokenizer.tokenize(text).split()
        text_segmented = [elem for elem in text_segmented if elem not in stopwords]
        return ' '.join(text_segmented)

reviews = []
score_comment = []

for i, text in enumerate(comment_score['comment']):
    try:
        score_comment.append(text['score'])
    except:
        score_comment.append(-100)
    a = prepocess(text['comment'])
    reviews.append(tokenize(a,stopwords))

X.shape

In [11]:
df

,comment,score


In [12]:
df[df['score'] == -100]

,comment,score


In [13]:
df.drop

<bound method DataFrame.drop of Empty DataFrame
Columns: [comment, score]
Index: []>

df.loc[df.score == -100,'score'] = 8

In [14]:
df.iloc[843]

IndexError: single positional indexer is out-of-bounds

In [ ]:
df.iloc[1717]

In [ ]:
df.loc[df.score == -100,'score'][843] = 4

In [ ]:
df.loc[df.score == -100,'score'][1717] = 9

In [ ]:
df.iloc[843].score = 4

In [ ]:
df.iloc[1717].score = 9

In [ ]:
df.loc[df.score == -100,'score']

In [ ]:
int(df['score'].iloc[0]) >= 6.0

In [ ]:
df['score'].values

In [ ]:
for i,ele in enumerate(df['score'].values):
    if (float(ele) >= 6.0):
        df.iloc[i].score = 1
    else:
        df.iloc[i].score = 0

In [ ]:
df

In [ ]:
import matplotlib 
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [ ]:
numpy_label = df['score'].values

In [ ]:
plt.xlabel("number")
plt.ylabel("name_class")
plt.title("su phan bo cua cac class")
plt.hist(numpy_label,rwidth = 1,align ="left")
plt.savefig('demo.png')

In [ ]:
with open("/home/hieuvt/Projects/Crawl_comments/vietnamese-stopwords-dash.txt", encoding="utf8") as input_file:
    x = input_file.readlines()
    removetable = str.maketrans('', '', '\n')
    stopwords = [s.translate(removetable) for s in x]
def prepocess(text):
    return preprocessor.preprocess(text)
def tokenize(text, stopwords):
        text_segmented = ViTokenizer.tokenize(text).split()
        text_segmented = [elem for elem in text_segmented if elem not in stopwords]
        return ' '.join(text_segmented)

In [ ]:
reviews = []
score_comment = []

In [ ]:
for i, text in enumerate(df['comment']):
#     score_comment.append(text['score'])
    a = prepocess(text)
    reviews.append(tokenize(a,stopwords))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(min_df=5,max_df= 0.8,sublinear_tf=True)
tf.fit(reviews)
X = tf.transform(reviews)

In [ ]:
Y = X.todense().A

In [ ]:
type(Y)

In [ ]:
X_label = df['score'].values

In [ ]:
type(X_label)

In [ ]:
X_train,X_test, Y_train, Y_test = train_test_split(Y,X_label,test_size = 0.2, random_state = 42)

In [ ]:
import numpy as np
np.unique(Y_train)

In [ ]:
Y_train.shape

In [ ]:
test = pd.DataFrame()
test['y'] = Y_train

In [ ]:
test.y.isnull().mean()

In [ ]:
test.y.unique()

In [ ]:
Y_train = Y_train.astype(int)

In [ ]:
X_train

In [ ]:
Y_train

In [ ]:
clf = svm.SVC(verbose=True, kernel='linear', probability=True, random_state=0, cache_size=2000, class_weight='balanced')
clf.fit(X_train,Y_train)
print(clf.n_support_)

In [ ]:
Y_pred = clf.predict(X_test)

In [ ]:
Y_test = Y_test.astype(int)

In [ ]:
precision, recall, fscore, support = score(Y_test, Y_pred)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))